NP-CHUNKING (I-O-B tagging) with LLMs.  

Stephan Raaijmakers, LUCL, 28.03.2025

In [1]:
!pip install langchain_huggingface

In [2]:
from transformers import AutoModelForCausalLM , AutoTokenizer, pipeline
from huggingface_hub import InferenceClient
import sys
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import re

In [3]:
from langchain_core.messages import (
    HumanMessage,
    SystemMessage,
)

In [4]:
from langchain_huggingface.llms import HuggingFacePipeline

In [10]:
from langchain_huggingface import ChatHuggingFace,HuggingFaceEndpoint

In [6]:
from google.colab import userdata
HUGGINGFACE_TOKEN=userdata.get('HF_TOKEN')

Notice you will need a Huggingface PRO account ($9/mo) for the InferenceClient on non-free tier models.

In [7]:
def generate_iob_tags_ic(lines, output_file):
    print("Generating IOB tags...")
    #HUGGINGFACE_TOKEN=os.environ["HUGGINGFACE_TOKEN"] # run first: export HUGGINGFACE_TOKEN="..." in shell
    outp=open(output_file,"w")

    client = InferenceClient(
                api_key=HUGGINGFACE_TOKEN,
            )
    for i in tqdm(range(len(lines))):
        sentence=' '.join(lines[i])

        messages = [
	                {
		            "role": "user",
		            "content": "Assign IOB tags for NP-chunking to the following sentence. Check that no taggings are repeated in your output. Every word in the sentence should have just one tag. If you have multiple options just output one. Tags are limited to O, B-NP, I-NP. Output format is: word/tag, per separate word. Example: The sandwich was good. Output: The/B-NP sandwich/I-NP was/O good/O. Sentence: "+sentence
	                }
                    ]

        completion = client.chat.completions.create(
                 #model="meta-llama/Meta-Llama-3-8B-Instruct",
                 model="mistralai/Mistral-7B-Instruct-v0.2",
	             messages=messages,
	             max_tokens=500,
                 temperature=0.0,
            )

        iob_tagged=completion.choices[0].message.content
        iob_tagged=re.sub("\n"," ",iob_tagged)
        iob_tagged=re.sub("Here is the output: ","",iob_tagged)
        outp.write("Sentence:%s\nTags:%s\n"%(sentence,iob_tagged))

In [24]:
llm = HuggingFaceEndpoint(
    #repo_id="HuggingFaceH4/zephyr-7b-beta",
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
    temperature=0.0
)

In [25]:
def generate_iob_tags_hf(lines, output_file):
    print("Generating IOB tags...")
    #HUGGINGFACE_TOKEN=os.environ["HUGGINGFACE_TOKEN"] # run first: export HUGGINGFACE_TOKEN="..." in shell
    outp=open(output_file,"w")

    chat_model = ChatHuggingFace(llm=llm)

    for i in tqdm(range(len(lines))):
        sentence=' '.join(lines[i])

        messages = [
        SystemMessage(content="Assign IOB tags for NP-chunking to the following sentence. Check that no taggings are repeated in your output. Every word in the sentence should have just one tag. If you have multiple options just output one. Tags are limited to O, B-NP, I-NP. Output format is: word/tag, per separate word. Example: The sandwich was good. Output: The/B-NP sandwich/I-NP was/O good/O"),
        HumanMessage(
        content="Sentence:"+sentence
          ),
        ]

        completion = chat_model.invoke(messages)

        iob_tagged=completion.content
        iob_tagged=re.sub("\n"," ",iob_tagged)
        iob_tagged=re.sub("Here is the output: ","",iob_tagged)
        outp.write("Sentence:%s\nTags:%s\n"%(sentence,iob_tagged))

In [26]:

def main(input_file, output_file):
    with open(input_file,"r") as f:
        lines = [z for z in [x.rstrip().split(" ") for x in f.readlines()]]
    generate_iob_tags_hf(lines, output_file)
    #generate_iob_tags_ic(lines, output_file)
    print("See ", output_file)

In [27]:
main("det.txt", "iob.out")

Generating IOB tags...


100%|██████████| 8/8 [00:01<00:00,  7.94it/s]

See  iob.out
